In [1]:
## This file implements neural networks and random forest on p0017SpresabsSTCC_qual.
## Due to the imbalanced data and limited cases for class 2, we implement the over-sampling method.
## For fully-connected neural networks, the accuracy is 91.37% for over-sampling data and 88.83% with dropout and regularization.
## For random forest, the accuracy is 96.45% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 93.01% for over-sampling data.

In [1]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p0017SpresabsSTCC_qual.csv')
df.shape

(255, 148)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
5      0
6      0
7      0
8      1
9      0
10     0
11     1
12     0
13     0
14     0
15     0
16     1
17     1
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25     0
26     0
27     0
28     0
29     1
      ..
225    0
226    0
227    1
228    0
229    1
230    0
231    0
232    0
233    0
234    0
235    0
236    0
237    0
238    0
239    0
240    0
241    0
242    0
243    0
244    1
245    0
246    0
247    0
248    0
249    0
250    1
251    0
252    0
253    0
254    0
Name: pheno, Length: 255, dtype: int64

In [5]:
df['pheno'].value_counts()

0    218
1     35
2      2
Name: pheno, dtype: int64

In [6]:
df.head()

,id,TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT,TTTTCCAGTAAT,TTTTAATACATAT,TTTTAAATATTATAA,TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC,...,group_3904,group_426,group_475,group_6375,group_7822,group_8071,group_8913,ST,CC,pheno
0,107,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,5,5,0
1,109,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,115,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,5,5,1
3,120335,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,5,5,0
4,120337,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,5,5,0


In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(255, 147)

In [9]:
df_clean.head()

,TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT,TTTTCCAGTAAT,TTTTAATACATAT,TTTTAAATATTATAA,TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC,TTCCATCGAATCAC,...,group_3904,group_426,group_475,group_6375,group_7822,group_8071,group_8913,ST,CC,pheno
0,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,5,5,0
1,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,5,5,1
3,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,5,5,0
4,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,5,5,0


In [10]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 146) (255,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 218), (1, 218), (2, 218)]


Using TensorFlow backend.


In [12]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [13]:
############# Fully-Connected Neural Network ################

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [27]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [28]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 457 samples, validate on 197 samples
Epoch 1/100
457/457 [==============================] - 0s 250us/step - loss: 11.7254 - accuracy: 0.4902 - val_loss: 9.9353 - val_accuracy: 0.4467
Epoch 2/100
457/457 [==============================] - 0s 72us/step - loss: 5.6036 - accuracy: 0.5580 - val_loss: 4.1643 - val_accuracy: 0.6599
Epoch 3/100
457/457 [==============================] - 0s 74us/step - loss: 1.4502 - accuracy: 0.6543 - val_loss: 1.1575 - val_accuracy: 0.5127
Epoch 4/100
457/457 [==============================] - 0s 81us/step - loss: 0.9215 - accuracy: 0.6411 - val_loss: 0.7348 - val_accuracy: 0.5330
Epoch 5/100
457/457 [==============================] - 0s 76us/step - loss: 0.6478 - accuracy: 0.7068 - val_loss: 0.5086 - val_accuracy: 0.7766
Epoch 6/100
457/457 [==============================] - 0s 74us/step - loss: 0.4830 - accuracy: 0.8140 - val_loss: 0.5271 - val_accuracy: 0.7919
Epoch 7/100
457/457 [==============================] - 0s 72us/step - loss: 0.4156 - acc

457/457 [==============================] - 0s 80us/step - loss: 0.1603 - accuracy: 0.9475 - val_loss: 0.2355 - val_accuracy: 0.8985
Epoch 58/100
457/457 [==============================] - 0s 81us/step - loss: 0.1562 - accuracy: 0.9540 - val_loss: 0.2314 - val_accuracy: 0.9137
Epoch 59/100
457/457 [==============================] - 0s 66us/step - loss: 0.1582 - accuracy: 0.9453 - val_loss: 0.2388 - val_accuracy: 0.9137
Epoch 60/100
457/457 [==============================] - 0s 60us/step - loss: 0.3189 - accuracy: 0.9125 - val_loss: 0.6350 - val_accuracy: 0.8832
Epoch 61/100
457/457 [==============================] - 0s 59us/step - loss: 0.3901 - accuracy: 0.9015 - val_loss: 0.2829 - val_accuracy: 0.8629
Epoch 62/100
457/457 [==============================] - 0s 57us/step - loss: 0.2053 - accuracy: 0.9453 - val_loss: 0.2572 - val_accuracy: 0.9086
Epoch 63/100
457/457 [==============================] - 0s 57us/step - loss: 0.1804 - accuracy: 0.9497 - val_loss: 0.3466 - val_accuracy: 0.898

In [30]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

197/197 [==============================] - 0s 101us/step
over-sampling test accuracy: 91.37%


In [31]:
#### add dropout and regularizer
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [32]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 457 samples, validate on 197 samples
Epoch 1/100
457/457 [==============================] - 0s 427us/step - loss: 11.8559 - accuracy: 0.3392 - val_loss: 10.6516 - val_accuracy: 0.3046
Epoch 2/100
457/457 [==============================] - 0s 90us/step - loss: 11.4729 - accuracy: 0.4004 - val_loss: 9.0253 - val_accuracy: 0.5076
Epoch 3/100
457/457 [==============================] - 0s 81us/step - loss: 9.6839 - accuracy: 0.5142 - val_loss: 7.6035 - val_accuracy: 0.5736
Epoch 4/100
457/457 [==============================] - 0s 86us/step - loss: 8.9622 - accuracy: 0.5295 - val_loss: 6.9814 - val_accuracy: 0.5431
Epoch 5/100
457/457 [==============================] - 0s 83us/step - loss: 8.1847 - accuracy: 0.5033 - val_loss: 6.1899 - val_accuracy: 0.5431
Epoch 6/100
457/457 [==============================] - 0s 83us/step - loss: 7.5786 - accuracy: 0.5274 - val_loss: 5.6494 - val_accuracy: 0.5431
Epoch 7/100
457/457 [==============================] - 0s 99us/step - loss: 6.9898 - a

457/457 [==============================] - 0s 92us/step - loss: 1.6266 - accuracy: 0.7615 - val_loss: 0.6729 - val_accuracy: 0.8629
Epoch 58/100
457/457 [==============================] - 0s 82us/step - loss: 1.5060 - accuracy: 0.7505 - val_loss: 0.7213 - val_accuracy: 0.8477
Epoch 59/100
457/457 [==============================] - 0s 76us/step - loss: 1.3631 - accuracy: 0.7637 - val_loss: 0.5756 - val_accuracy: 0.8934
Epoch 60/100
457/457 [==============================] - 0s 71us/step - loss: 1.5783 - accuracy: 0.7352 - val_loss: 0.7279 - val_accuracy: 0.8071
Epoch 61/100
457/457 [==============================] - 0s 85us/step - loss: 1.5349 - accuracy: 0.7593 - val_loss: 0.7424 - val_accuracy: 0.8782
Epoch 62/100
457/457 [==============================] - 0s 66us/step - loss: 1.7797 - accuracy: 0.7309 - val_loss: 0.6810 - val_accuracy: 0.8680
Epoch 63/100
457/457 [==============================] - 0s 65us/step - loss: 1.8839 - accuracy: 0.7265 - val_loss: 0.8435 - val_accuracy: 0.883

In [34]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

197/197 [==============================] - 0s 70us/step
over-sampling test accuracy: 88.83%


In [23]:
############## Random Forest ##############

In [24]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [25]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 96.45%


In [26]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.92473118 0.92473118 0.92307692 0.94444444 0.93333333]
0.9300634132892197
